In [23]:
import pandas as pd

# Загрузка данных
data = pd.read_csv('data_for_spb_hakaton_entities1-Table 1.csv', sep=';')
history = pd.read_csv('history-Table 1.csv', sep=';', index_col=None)
sprints = pd.read_csv('sprints-Table 1.csv', sep=';', index_col=None)

In [25]:
# Сдвиг колонок влево, чтобы "entity_id" стало первой колонкой
history = history.reset_index()  # Сброс индекса
columns = history.columns.tolist()  # Получаем текущий список колонок
columns = columns[1:] + columns[:1]  # Перемещаем первую колонку в конец
history.columns = columns  # Переупорядочиваем названия колонок
history.head()

,entity_id,history_property_name,history_date,history_version,history_change_type,history_change,index
0,94297.0,Время решения 3ЛП ФАКТ,09.10.2024 11:17,1.0,FIELD_CHANGED,<empty> -> 2024-09-10 11:17:06.680223,NaN
1,94297.0,Время решения (ФАКТ),09.10.2024 11:17,1.0,FIELD_CHANGED,<empty> -> 2024-09-10 11:17:06.680223,NaN
2,94297.0,Исполнитель,7/13/23 11:07,1.0,FIELD_CHANGED,user409017mail@mail.com -> user408045mail@mail...,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,94297.0,Исполнитель,7/21/23 11:06,3.0,FIELD_CHANGED,user4080458@mail.com -> user4091341@mail.com,NaN


In [27]:
if 'index' in history.columns:
    history.drop(columns=['index'], inplace=True)

In [29]:

# Разделение entity_ids в таблице sprints
sprints['entity_ids'] = sprints['entity_ids'].apply(lambda x: x.strip('{}').split(',') if pd.notnull(x) else [])
sprints_expanded = sprints.explode('entity_ids')
sprints_expanded['entity_id'] = sprints_expanded['entity_ids'].astype(float)

# Удаляем строки с некорректным типом в entity_id из history
history = history[history['entity_id'].apply(lambda x: isinstance(x, (int, float)))]

# Приведение entity_id в обоих таблицах к одному типу (float)
data['entity_id'] = data['entity_id'].astype(float)
history['entity_id'] = history['entity_id'].astype(float)

# Соединение data и sprints
data_sprints = pd.merge(data, sprints_expanded, on='entity_id', how='left')

# Соединение data_sprints с history
final_table = pd.merge(data_sprints, history, on='entity_id', how='left')

# Просмотр результата
print(final_table.head())

# Если требуется сохранение в файл
final_table.to_csv('merged_data.csv', index=False)

# TODO: DATA = FINALTABLE.COPY()


   entity_id                 area    type   status   state priority  \
0    94297.0  Система.Таск-трекер  Дефект  Закрыто  Normal  Средний   
1    94297.0  Система.Таск-трекер  Дефект  Закрыто  Normal  Средний   
2    94297.0  Система.Таск-трекер  Дефект  Закрыто  Normal  Средний   
3    94297.0  Система.Таск-трекер  Дефект  Закрыто  Normal  Средний   
4    94297.0  Система.Таск-трекер  Дефект  Закрыто  Normal  Средний   

  ticket_number                                               name  \
0     PPTS-1965  [FE] Бэклог. Кастомизация колонок. Кастомизаци...   
1     PPTS-1965  [FE] Бэклог. Кастомизация колонок. Кастомизаци...   
2     PPTS-1965  [FE] Бэклог. Кастомизация колонок. Кастомизаци...   
3     PPTS-1965  [FE] Бэклог. Кастомизация колонок. Кастомизаци...   
4     PPTS-1965  [FE] Бэклог. Кастомизация колонок. Кастомизаци...   

                  create_date created_by  ...  \
0  2023-03-16 16:59:00.000000      А. К.  ...   
1  2023-03-16 16:59:00.000000      А. К.  ...   
2  20

In [31]:
# Выбор конкретного спринта
selected_sprint_entity_ids = sprints.iloc[0]['entity_ids']  # Уже является списком

# Фильтрация задач, относящихся к выбранному спринту, со статусом "Создано"
tasks_in_sprint = data[
    (data['entity_id'].isin(selected_sprint_entity_ids)) &
    (data['status'] == 'Создано')
]

# Суммирование estimation и деление на 3600
first_metric_sum = tasks_in_sprint['estimation'].sum() / 3600  # Перевод в часы

# Создание отдельного столбца с рассчитанным показателем для каждой строки (если нужен общий результат - только sum)
data['first_metric'] = 0  # Инициализация
data.loc[
    data['entity_id'].isin(selected_sprint_entity_ids),
    'first_metric'
] = first_metric_sum

In [32]:
# Группируем задачи по sprintName и рассчитываем метрику "К выполнению"
def calculate_metric_per_sprint(df, sprint_name):
    # Фильтрация задач, относящихся к указанному спринту и имеющих статус "Создано"
    tasks_in_sprint = df[
        (df['sprint_name'] == sprint_name) &
        (df['status'] == 'Создано')
    ]
    # Суммирование estimation для задач спринта и деление на 3600
    metric = tasks_in_sprint['estimation'].sum() / 3600
    return metric

# Применяем расчёт метрики по каждому уникальному спринту
sprint_metrics = {}
for sprint_name in data['sprint_name'].unique():
    sprint_metrics[sprint_name] = calculate_metric_per_sprint(data, sprint_name)

# Добавляем результат в таблицу
data['second_metric'] = data['sprint_name'].map(sprint_metrics)


In [40]:
import pandas as pd

# Преобразуем столбец sprint_start_date в datetime в DataFrame sprints
sprints['sprint_start_date'] = pd.to_datetime(sprints['sprint_start_date'], errors='coerce')

# Функция для разделения задач по времени
def split_tasks_by_time(df, sprint_start_date):
    # Устанавливаем границу времени - два дня после начала спринта
    two_days_after_start = sprint_start_date + pd.Timedelta(days=2)
    
    # Преобразуем history_date в datetime
    df['history_date'] = pd.to_datetime(df['history_date'], errors='coerce')
    
    # Задачи на момент начала спринта (до начала)
    early_tasks_df = df[
        (df['history_date'].isna() | (df['history_date'] <= sprint_start_date)) &
        (df['type'] != 'Дефект')  # Исключаем дефекты
    ]
    
    # Задачи, добавленные в первые два дня
    late_tasks_df = df[
        (df['history_date'] > sprint_start_date) &
        (df['history_date'] <= two_days_after_start) &
        (df['type'] != 'Дефект')  # Исключаем дефекты
    ]
    
    return early_tasks_df, late_tasks_df

# Функция для подсчета backlog_change для спринта
def calculate_backlog_change(data, sprint):
    # Убедимся, что sprint_start_date в формате datetime
    sprint_start_date = sprint['sprint_start_date']
    
    # Разделяем задачи на ранние (до начала спринта) и поздние (первые два дня)
    early_tasks_df, late_tasks_df = split_tasks_by_time(data, sprint_start_date)

    # Считаем сумму оценок
    early_sum = early_tasks_df['estimation'].sum() / 3600  # Переводим секунды в часы
    late_sum = late_tasks_df['estimation'].sum() / 3600  # Переводим секунды в часы

    # Исправление расчета для случаев, когда early_sum == 0
    if early_sum > 0:
        backlog_change_pct = (late_sum * 100) / early_sum
    elif late_sum > 0:
        backlog_change_pct = 100.0  # Весь бэклог был добавлен поздно
    else:
        backlog_change_pct = 0.0  # Нет задач в бэклоге

    return round(backlog_change_pct, 1)

# Рассчитываем backlog_change для каждого спринта
backlog_metrics = []
for _, sprint in sprints.iterrows():
    backlog_change_pct = calculate_backlog_change(data, sprint)
    backlog_metrics.append({
        'sprint_name': sprint['sprint_name'],  # Убедитесь, что есть колонка 'sprint_name'
        'backlog_change_percentage': backlog_change_pct
    })

# Создаем итоговый DataFrame с метриками
backlog_metrics_df = pd.DataFrame(backlog_metrics)

# Просмотр результата
print(backlog_metrics_df)


                         sprint_name  backlog_change_percentage
0  Спринт 2024.3.1.NPP Shared Sprint                        2.8
1  Спринт 2024.3.2.NPP Shared Sprint                        0.0
2  Спринт 2024.3.3.NPP Shared Sprint                        3.5
3  Спринт 2024.3.4.NPP Shared Sprint                        0.0
4  Спринт 2024.3.5.NPP Shared Sprint                        0.0
5  Спринт 2024.3.6.NPP Shared Sprint                        0.6


In [41]:
import pandas as pd

# Преобразуем столбец sprint_start_date и sprint_end_date в datetime
sprints['sprint_start_date'] = pd.to_datetime(sprints['sprint_start_date'], errors='coerce')
sprints['sprint_end_date'] = pd.to_datetime(sprints['sprint_end_date'], errors='coerce')

# Функция для разделения задач по времени относительно дня спринта
def split_tasks_by_day(df, current_date):
    # Преобразуем history_date в datetime
    df['history_date'] = pd.to_datetime(df['history_date'], errors='coerce')
    
    # Задачи до текущей даты
    early_tasks_df = df[
        (df['history_date'].isna() | (df['history_date'] <= current_date)) &
        (df['type'] != 'Дефект')  # Исключаем дефекты
    ]
    
    # Задачи, добавленные до текущей даты
    added_tasks_df = df[
        (df['history_date'] > current_date) &
        (df['type'] != 'Дефект')  # Исключаем дефекты
    ]
    
    return early_tasks_df, added_tasks_df

# Функция для подсчета backlog_change для определённого дня спринта
def calculate_daily_backlog_change(data, sprint, current_date):
    # Разделяем задачи на те, что существовали до текущей даты, и те, что добавились
    early_tasks_df, added_tasks_df = split_tasks_by_day(data, current_date)

    # Считаем сумму оценок
    early_sum = early_tasks_df['estimation'].sum() / 3600  # Переводим секунды в часы
    added_sum = added_tasks_df['estimation'].sum() / 3600  # Переводим секунды в часы

    # Исправление расчета для случаев, когда early_sum == 0
    if early_sum > 0:
        backlog_change_pct = (added_sum * 100) / early_sum
    elif added_sum > 0:
        backlog_change_pct = 100.0  # Весь бэклог был добавлен поздно
    else:
        backlog_change_pct = 0.0  # Нет задач в бэклоге

    return round(backlog_change_pct, 1)

# Рассчитываем backlog_change для каждого дня в каждом спринте
daily_backlog_metrics = []
for _, sprint in sprints.iterrows():
    sprint_name = sprint['sprint_name']
    sprint_start_date = sprint['sprint_start_date']
    sprint_end_date = sprint['sprint_end_date']
    
    # Перебор дней спринта
    current_date = sprint_start_date
    while current_date <= sprint_end_date:
        backlog_change_pct = calculate_daily_backlog_change(data, sprint, current_date)
        daily_backlog_metrics.append({
            'sprint_name': sprint_name,
            'day': current_date,
            'backlog_change_percentage': backlog_change_pct
        })
        current_date += pd.Timedelta(days=1)  # Переходим к следующему дню

# Создаем итоговый DataFrame с ежедневными метриками
daily_backlog_metrics_df = pd.DataFrame(daily_backlog_metrics)


In [45]:
daily_backlog_metrics_df.head()

,sprint_name,day,backlog_change_percentage
0,Спринт 2024.3.1.NPP Shared Sprint,2024-07-03 19:00:00,63.0
1,Спринт 2024.3.1.NPP Shared Sprint,2024-07-04 19:00:00,60.5
2,Спринт 2024.3.1.NPP Shared Sprint,2024-07-05 19:00:00,58.5
3,Спринт 2024.3.1.NPP Shared Sprint,2024-07-06 19:00:00,58.5
4,Спринт 2024.3.1.NPP Shared Sprint,2024-07-07 19:00:00,58.5


In [60]:
# Подготовка данных для расчёта

# Преобразование дат начала и окончания спринта
sprints['sprint_start_date'] = pd.to_datetime(sprints['sprint_start_date'])
sprints['sprint_end_date'] = pd.to_datetime(sprints['sprint_end_date'])

# Выбор конкретного спринта для анализа
selected_sprint = sprints.iloc[0]  # Например, первый спринт
sprint_start_date = selected_sprint['sprint_start_date']
sprint_end_date = selected_sprint['sprint_end_date']
two_days_after_start = sprint_start_date + pd.Timedelta(days=2)

# Вычисляем startOfSprint: задачи, которые уже были в спринте до его начала
start_of_sprint_tasks = data[
    (data['history_date'].isna() | 
     (pd.to_datetime(data['history_date']) <= sprint_start_date)) &
    (data['status'] != "Дефект")  # Исключаем дефекты
]
start_of_sprint_sum = start_of_sprint_tasks['estimation'].sum() / 3600  # Переводим секунды в часы

# Вычисляем twoDaysAfterStartOfSprint: задачи, добавленные в первые два дня спринта
tasks_added_in_two_days = data[
    (pd.to_datetime(data['history_date']) > sprint_start_date) &
    (pd.to_datetime(data['history_date']) <= two_days_after_start) &
    (data['status'] != "Дефект")  # Исключаем дефекты
]
two_days_after_start_sum = tasks_added_in_two_days['estimation'].sum() / 3600  # Переводим секунды в часы

# Рассчитываем процентное изменение бэклога
if start_of_sprint_sum > 0:
    backlog_change_percentage = (two_days_after_start_sum * 100) / start_of_sprint_sum
else:
    backlog_change_percentage = 0

# Добавляем результат в DataFrame
data['backlog_change_percentage'] = backlog_change_percentage



In [61]:
data['backlog_change_percentage'].unique()

array([0.])

In [35]:
%pip install timedelta

  Preparing metadata (setup.py) ... done
  Created wheel for timedelta: filename=timedelta-2020.12.3-py3-none-any.whl size=1555 sha256=c7ec5b442e27869423103a70257f2cefd051b84bdde4bbee9caa8ac863d8f15b
  Stored in directory: /home/denis/.cache/pip/wheels/be/c0/1f/843bd73679dc7dfb26758c4effc6e273d3eebf2da2a35082b1
Successfully built timedelta
Note: you may need to restart the kernel to use updated packages.


In [15]:
columns_to_drop = ['state', 'name', 'updated_by', 'parent_ticket_id', 'assignee', 'owner', 'due_date', 'rank', 'spent', 'workgroup', 'history_version', 'history_change_type', 'history_change', 'Столбец1']

In [16]:
data = data.drop(columns=columns_to_drop)

In [17]:
data.columns

Index(['entity_id', 'area', 'type', 'status', 'priority', 'ticket_number',
       'create_date', 'created_by', 'update_date', 'estimation', 'resolution',
       'sprint_name', 'sprint_status', 'sprint_start_date', 'sprint_end_date',
       'entity_ids', 'history_property_name', 'history_date'],
      dtype='object')

In [22]:
data['priority'].unique()

array(['Средний', 'Критический', 'Высокий', 'Низкий'], dtype=object)